<a href="https://colab.research.google.com/github/nahyunryou/ML_Project/blob/main/Catboost2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 27 kB/s 


In [2]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 4.1 MB/s 
     |████████████████████████████████| 81 kB 10.1 MB/s 
     |████████████████████████████████| 209 kB 61.3 MB/s 
     |████████████████████████████████| 78 kB 8.2 MB/s 
     |████████████████████████████████| 50 kB 3.7 MB/s 
     |████████████████████████████████| 147 kB 56.0 MB/s 
     |████████████████████████████████| 112 kB 61.7 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=3f8e28e5d82684144b7da019130d9133e48dbb6157358795d8b5d313add42d81
  Stored in directory: /root/.cache/pip/wheels/7f/1a/65/84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built pyperclip


In [3]:
import warnings
warnings.filterwarnings('ignore')

import glob
import pandas as pd
import numpy as np

from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.metrics import log_loss
from sklearn.preprocessing import OneHotEncoder
import random

import optuna
from optuna.samplers import TPESampler

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
train = pd.read_csv('/content/drive/MyDrive/ml_project/train.csv')
train = train.drop(['index'], axis=1)
train.fillna('NAN',inplace=True)

test = pd.read_csv('/content/drive/MyDrive/ml_project/test.csv')
test = test.drop(['index'],axis=1)
test.fillna('NAN',inplace=True)

submit = pd.read_csv('/content/drive/MyDrive/ml_project/sample_submission.csv')

In [18]:
edu_dict = {'Lower secondary':1,
 'Secondary / secondary special':2,
 'Incomplete higher':3,
 'Higher education':4,
 'Academic degree':5}

train['edu_type'].replace(edu_dict,inplace=True)
test['edu_type'].replace(edu_dict,inplace=True)

In [19]:
house_dict = {'With parents':3,
 'Co-op apartment':2,
 'Municipal apartment':1,
 'Rented apartment':4,
 'Office apartment':5,
 'House / apartment':6}

train['house_type'].replace(house_dict,inplace=True)
test['house_type'].replace(house_dict,inplace=True)

In [20]:
#train['CODE'] = train['gender']+train['DAYS_BIRTH'].apply(str)+train['income_total'].apply(str)+train['income_type'].apply(str)
#test['CODE'] = test['gender']+test['DAYS_BIRTH'].apply(str)+test['income_total'].apply(str)+test['income_type'].apply(str)

In [21]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26457 entries, 0 to 26456
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   gender         26457 non-null  object 
 1   car            26457 non-null  object 
 2   reality        26457 non-null  object 
 3   child_num      26457 non-null  int64  
 4   income_total   26457 non-null  float64
 5   income_type    26457 non-null  object 
 6   edu_type       26457 non-null  int64  
 7   family_type    26457 non-null  object 
 8   house_type     26457 non-null  int64  
 9   DAYS_BIRTH     26457 non-null  int64  
 10  DAYS_EMPLOYED  26457 non-null  int64  
 11  FLAG_MOBIL     26457 non-null  int64  
 12  work_phone     26457 non-null  int64  
 13  phone          26457 non-null  int64  
 14  email          26457 non-null  int64  
 15  occyp_type     26457 non-null  object 
 16  family_size    26457 non-null  float64
 17  begin_month    26457 non-null  float64
 18  credit

In [22]:
#train.to_csv(f'/content/drive/MyDrive/ml_project/code_column.csv',index=False)

In [23]:
#Optuna용 Train셋
X = train.drop(['credit'],axis=1)
y = train['credit']
X_test = test.copy()

In [26]:
def objective(trial):
  param = {
      "random_state":42,
      'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.3),
      'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
      "n_estimators":trial.suggest_int("n_estimators", 1000, 10000),
      "max_depth":trial.suggest_int("max_depth", 4, 16),
      'random_strength' :trial.suggest_int('random_strength', 0, 100),
      "colsample_bylevel":trial.suggest_float("colsample_bylevel", 0.4, 1.0),
      "l2_leaf_reg":trial.suggest_float("l2_leaf_reg",1e-8,3e-5),
      "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
      "max_bin": trial.suggest_int("max_bin", 200, 500),
      'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
  }

  X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size=0.2)
  
  #cat_features =[0,1,2,5,6,7,8,15,18]
  cat_features =[0,1,2,5,6,7,8,15]
  cat = CatBoostClassifier(**param)
  cat.fit(X_train, y_train,
          eval_set=[(X_train, y_train), (X_valid,y_valid)],
          early_stopping_rounds=35,cat_features=cat_features,
          verbose=100)
  cat_pred = cat.predict_proba(X_valid)
  log_score = log_loss(y_valid, cat_pred)

  return log_score

In [27]:
sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name = 'cat_parameter_opt',
    direction = 'minimize',
    sampler = sampler,
)
study.optimize(objective, n_trials=10)
print("Best Score:",study.best_value)
print("Best trial",study.best_trial.params)

[I 2022-12-02 09:10:42,747] A new study created in memory with name: cat_parameter_opt


0:	learn: 1.0829143	test: 1.0829143	test1: 1.0829492	best: 1.0829492 (0)	total: 58.2ms	remaining: 7m 21s
100:	learn: 0.8827588	test: 0.8827588	test1: 0.8821878	best: 0.8821878 (100)	total: 887ms	remaining: 1m 5s
200:	learn: 0.8825838	test: 0.8825838	test1: 0.8819301	best: 0.8819301 (200)	total: 1.7s	remaining: 1m 2s
300:	learn: 0.8825836	test: 0.8825836	test1: 0.8819277	best: 0.8819277 (300)	total: 2.5s	remaining: 1m
400:	learn: 0.8825836	test: 0.8825836	test1: 0.8819277	best: 0.8819277 (400)	total: 3.31s	remaining: 59.4s
500:	learn: 0.8825836	test: 0.8825836	test1: 0.8819277	best: 0.8819277 (500)	total: 4.13s	remaining: 58.5s
600:	learn: 0.8825836	test: 0.8825836	test1: 0.8819277	best: 0.8819277 (600)	total: 4.94s	remaining: 57.4s
700:	learn: 0.8825836	test: 0.8825836	test1: 0.8819277	best: 0.8819277 (694)	total: 5.75s	remaining: 56.5s


[I 2022-12-02 09:10:49,568] Trial 0 finished with value: 0.8819276502263708 and parameters: {'learning_rate': 0.03574712922600244, 'bagging_temperature': 63.512210106407046, 'n_estimators': 7588, 'max_depth': 11, 'random_strength': 15, 'colsample_bylevel': 0.49359671220172163, 'l2_leaf_reg': 1.7519275289243016e-06, 'min_child_samples': 88, 'max_bin': 380, 'od_type': 'IncToDec'}. Best is trial 0 with value: 0.8819276502263708.


Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.8819276502
bestIteration = 761

Shrink model to first 762 iterations.
0:	learn: 0.9617141	test: 0.9617141	test1: 0.9628902	best: 0.9628902 (0)	total: 51.7ms	remaining: 2m 30s
100:	learn: 0.7184380	test: 0.7213719	test1: 0.7968245	best: 0.7965527 (99)	total: 4.41s	remaining: 2m 2s
200:	learn: 0.6564633	test: 0.6606508	test1: 0.7888928	best: 0.7880943 (195)	total: 8.71s	remaining: 1m 57s


[I 2022-12-02 09:11:02,143] Trial 1 finished with value: 0.7865609113744659 and parameters: {'learning_rate': 0.2708160864249968, 'bagging_temperature': 21.368329072358772, 'n_estimators': 2911, 'max_depth': 6, 'random_strength': 18, 'colsample_bylevel': 0.5825453457757226, 'l2_leaf_reg': 1.5747445384650815e-05, 'min_child_samples': 46, 'max_bin': 287, 'od_type': 'IncToDec'}. Best is trial 1 with value: 0.7865609113744659.


...

0:	learn: 1.0147991	test: 1.0151840	test1: 1.0196569	best: 1.0196569 (0)	total: 109ms	remaining: 3m 46s
100:	learn: 0.3897062	test: 0.5390542	test1: 0.7783462	best: 0.7735473 (94)	total: 25.3s	remaining: 8m 15s


[I 2022-12-02 09:45:01,972] Trial 8 finished with value: 0.7735473061548296 and parameters: {'learning_rate': 0.20441878352493792, 'bagging_temperature': 0.7742116473996251, 'n_estimators': 2076, 'max_depth': 13, 'random_strength': 76, 'colsample_bylevel': 0.7367663185416977, 'l2_leaf_reg': 2.3131305726837285e-05, 'min_child_samples': 52, 'max_bin': 357, 'od_type': 'IncToDec'}. Best is trial 2 with value: 0.7199445292409854.


Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.7735473062
bestIteration = 94

Shrink model to first 95 iterations.
0:	learn: 1.0921954	test: 1.0921987	test1: 1.0922274	best: 1.0922274 (0)	total: 36.9ms	remaining: 4m 8s
100:	learn: 0.8896817	test: 0.8903330	test1: 0.8924018	best: 0.8924018 (100)	total: 3.66s	remaining: 4m
200:	learn: 0.8657736	test: 0.8672753	test1: 0.8706707	best: 0.8706707 (200)	total: 8.2s	remaining: 4m 26s
300:	learn: 0.8593001	test: 0.8616754	test1: 0.8655927	best: 0.8655915 (298)	total: 12.4s	remaining: 4m 25s
400:	learn: 0.8547102	test: 0.8579786	test1: 0.8623071	best: 0.8623058 (397)	total: 16.7s	remaining: 4m 23s
500:	learn: 0.8486830	test: 0.8523824	test1: 0.8572179	best: 0.8572179 (500)	total: 20.8s	remaining: 4m 18s
600:	learn: 0.8406631	test: 0.8449076	test1: 0.8502122	best: 0.8502122 (600)	total: 25s	remaining: 4m 14s
700:	learn: 0.8318097	test: 0.8367046	test1: 0.8425779	best: 0.8425779 (700)	total: 30.2s	remaining: 4m 19s
800:	learn:

[I 2022-12-02 09:49:37,000] Trial 9 finished with value: 0.7494189255209155 and parameters: {'learning_rate': 0.01443340240633889, 'bagging_temperature': 0.0133572404119741, 'n_estimators': 6728, 'max_depth': 8, 'random_strength': 51, 'colsample_bylevel': 0.9445398843556558, 'l2_leaf_reg': 7.486273952174759e-06, 'min_child_samples': 44, 'max_bin': 427, 'od_type': 'IncToDec'}. Best is trial 2 with value: 0.7199445292409854.


Best Score: 0.7199445292409854
Best trial {'learning_rate': 0.027010527749605478, 'bagging_temperature': 0.2920433847181412, 'n_estimators': 5105, 'max_depth': 14, 'random_strength': 20, 'colsample_bylevel': 0.708540663048167, 'l2_leaf_reg': 1.7776512920172654e-05, 'min_child_samples': 9, 'max_bin': 382, 'od_type': 'IncToDec'}


In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds = []
for train_idx, valid_idx in skf.split(train, train['credit']):
  folds.append((train_idx,valid_idx))

In [28]:
random.seed(42)
cat_models={}

#cat_features =[0,1,2,5,6,7,8,15,18]
cat_features =[0,1,2,5,6,7,8,15]

for fold in range(5):
  print(f'===================================={fold+1}============================================')
  train_idx, valid_idx = folds[fold]
  X_train = train.drop(['credit'],axis=1).iloc[train_idx].values 
  X_valid = train.drop(['credit'],axis=1).iloc[valid_idx].values
  y_train = train['credit'][train_idx].values
  y_valid = train['credit'][valid_idx].values

  cat = CatBoostClassifier(**study.best_params)
  cat.fit(X_train, y_train,
          eval_set=[(X_train, y_train), (X_valid,y_valid)],
          early_stopping_rounds=35,cat_features=cat_features,
          verbose=100)
  cat_models[fold] = cat
  print(f'================================================================================\n\n')

====================================1============================================
0:	learn: 1.0864997	test: 1.0865203	test1: 1.0866841	best: 1.0866841 (0)	total: 51.4ms	remaining: 4m 22s
100:	learn: 0.8469242	test: 0.8510126	test1: 0.8567042	best: 0.8567042 (100)	total: 14.7s	remaining: 12m 5s
200:	learn: 0.8258134	test: 0.8327720	test1: 0.8399934	best: 0.8399934 (200)	total: 26.7s	remaining: 10m 50s
300:	learn: 0.8108420	test: 0.8224534	test1: 0.8309943	best: 0.8309943 (300)	total: 45.1s	remaining: 12m
400:	learn: 0.7931460	test: 0.8085316	test1: 0.8180089	best: 0.8180089 (400)	total: 1m 1s	remaining: 11m 59s
500:	learn: 0.6701081	test: 0.7472359	test1: 0.7852730	best: 0.7852730 (500)	total: 2m	remaining: 18m 27s
600:	learn: 0.4165960	test: 0.5797555	test1: 0.7469257	best: 0.7469164 (598)	total: 3m 46s	remaining: 28m 16s
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.746174632
bestIteration = 610

Shrink model to first 611 iterations.


===========================

In [29]:
submit.iloc[:,1:]=0
for fold in range(5):
  submit.iloc[:,1:] += cat_models[fold].predict_proba(test)/5

In [30]:
import datetime
now = datetime.datetime.now()
created_time = now.strftime('%m%d-%H%M')

submit.to_csv(f'/content/drive/MyDrive/ml_project/{created_time}_submit.csv',index=False)

In [31]:
submit.head(20)

,index,0,1,2
0,26457,0.035305,0.153982,0.810713
1,26458,0.163619,0.182191,0.654190
2,26459,0.089724,0.146344,0.763933
3,26460,0.125020,0.093029,0.781951
4,26461,0.112421,0.177834,0.709745
5,26462,0.049177,0.189581,0.761243
6,26463,0.526367,0.453444,0.020189
7,26464,0.142137,0.184005,0.673858
8,26465,0.053407,0.134080,0.812512
9,26466,0.046325,0.384666,0.569010
